In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import math

In [2]:
data = pd.read_csv('train.csv')

In [15]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]

(41000,)

In [19]:


w_1 = np.random.rand(10, 784)
w_2 = np.random.rand(10, 10)
b_1 = np.random.rand(10,1)
dw_1 = np.ones((10, 784))
dw_2 = np.ones((10, 10))
db_1 = np.ones(10)

def relu(x):
    return (x>0) * x
    
def drelu(x):
    return 1 * (x > 0)

def sigmoid(x):
    return 1/(1 + (math.e)**(-x))

def dsigmoid(x):
    return sigmoid(x) * (1-sigmoid(x))

def feedforward(w_1, w_2, b_1, z_0):
    a_0 = np.copy(z_0)
    for i in range(784):
      a_0[i] = sigmoid(z_0[i])
    z_1 = np.matmul(w_1, a_0)
    z_1 += b_1.reshape((10,))
    a_1 = np.copy(z_1)
    for i in range(10):
        a_1[i] = sigmoid(a_1[i])
    z_2 = np.matmul(w_2, a_1)
    a_2 = np.copy(z_2)
    for i in range(10):
      a_2[i] = sigmoid(a_2[i])
    return z_1,z_2,a_0,a_1,a_2

def cost(w_1, w_2, b_1, i):
    sum = 0
    temp = np.zeros(10)
    temp[int(Y_train[i])] = 1
    sum += np.sum((temp - feedforward(w_1, w_2, b_1, X_train[:, i])[4]) ** 2)
    return sum
  
print(X_train[:,0])

feedforward(w_1,w_2,b_1,X_train[:,0])

(array([0.03817719, 0.89140217, 0.93520323, 0.51359017, 0.32877831,
        0.816352  , 0.31499746, 0.12401083, 0.73545851, 0.73091685]),
 array([2.68648024, 2.89364687, 3.06811288, 2.82653723, 2.86297695,
        2.21332152, 2.60525119, 2.81523717, 2.46131702, 3.00961634]),
 array([0.50954314, 0.70917945, 0.71812972, 0.62564772, 0.58146209,
        0.69346142, 0.57810461, 0.53096304, 0.67600196, 0.67500644]),
 array([0.93622414, 0.94753148, 0.9555581 , 0.94409311, 0.94598561,
        0.90143943, 0.93119877, 0.94349368, 0.92138511, 0.95300668]))

In [36]:
def epoch(alpha,w_1, w_2, b_1):
    dw_1 = np.ones((10, 784))
    dw_2 = np.ones((10, 10))
    db_1 = np.ones(10) 
    da_2 = np.ones(10)
    da_1 = np.ones((1,10))
    dz_2 = np.ones((1,10))
    dz_1 = np.ones((1,10))
    for i in range(18999):
        z_1,z_2,a_0,a_1,a_2 = feedforward(w_1,w_2,b_1,X_train[:,i])
        y_2 = np.zeros((1,10)) 
        y_2[0][int(Y_train[i])] = 1
        da_2 = (a_2.reshape((1,10))-y_2)
        for x in range(10):
          dz_2[0][x] = da_2[0][x] * dsigmoid(z_2[x])

        dw_2 = np.matmul(dz_2.reshape((10,1)),a_2.reshape((1,10)))
        #dw_2 = np.matmul(a_1.reshape((10,1)), dz_2)
        da_1 = np.matmul(dz_2.reshape((1,10)),w_2)

        dz_1 = np.copy(da_1)
        for x in range(10):
            dz_1[0][x] = dsigmoid(z_1[x]) * dz_1[0][x]
        db_1 = (1/10)* dz_1.reshape(10,1)

        dw_1 = np.matmul(dz_1.reshape((10,1)), a_0.reshape((1,784)))
        w_1 -= alpha * dw_1
        w_2 -= alpha * dw_2
        b_1 -= alpha * db_1.reshape(10,1)
    return w_1,w_2,b_1

In [37]:
#training
for i in range(1):
    w_1, w_2, b_1 = epoch(0.001, w_1, w_2, b_1)
    print(cost(w_1,w_2,b_1,i))

[[-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]]
[[-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]]
[[-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]]
[[-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]]
[[-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]]
[[-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]]
[[-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]]
[[-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]]
[[-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]]
[[-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 

KeyboardInterrupt: 

In [35]:
#accuracy
print(b_1)
def accuracy(w_1,w_2,b_1):
    n = 0
    for i in range(41000):
        a_2 = feedforward(w_1,w_2,b_1,X_train[:,i])[3]
        max = 0
        index = 0
        for x in range(10):
            if a_2[x] > max:
                max = a_2[x]
                index = x
        if Y_train[i] == x:
            n += 1
    return n / 41000
print(accuracy(w_1,w_2,b_1))

[[-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]
 [-0.001]]
0.09990243902439025
